In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import kwat
import numpy as np
import pandas as pd

In [ ]:
tp_ge_sa = pd.read_csv(
    "../output/tpm/tpmsum_geneselected_sample.tsv", sep="\t", index_col=0
)

In [ ]:
pa = "../output/compare_difference/"

kwat.path.make(pa)

In [ ]:
def get_difference(nu_fe_sa, sa1_, sa2_):

    return nu_fe_sa.loc[:, sa2_].mean(1) - nu_fe_sa.loc[:, sa1_].mean(1)

In [ ]:
sa1_ = ["S7"]

sa2_ = ["S8"]

di1_ = get_difference(tp_ge_sa, sa1_, sa2_)

di1_.name = "Healthy: CD4 ({}) - TA ({})".format(",".join(sa2_), ",".join(sa1_))

te = "{{}}_{{}}_vs_{}_{}".format("".join(sa2_), "".join(sa1_))

In [ ]:
pe_ = ["CD4", "HLA-A", "HLA-B", "HLA-C", "HLA-D", "HLA-E"]

In [ ]:
for sa1_, sa2_ in [
    [["S1", "S3", "S5"], ["S2", "S4", "S6"]],
    [["S1"], ["S2"]],
    [["S3"], ["S4"]],
    [["S5"], ["S6"]],
]:

    di2_ = get_difference(tp_ge_sa, sa1_, sa2_)

    di2_.name = "CFS: CD4 ({}) - TA ({})".format(",".join(sa2_), ",".join(sa1_))

    ch_ = di2_ - di1_

    na = kwat.path.clean(te.format("".join(sa2_), "".join(sa1_)))

    ch_.sort_values().to_csv("{}{}.tsv".format(pa, na), sep="\t")

    an_po_pa = pd.DataFrame(
        {
            di2_.name: kwat.array.log_with_negative(di2_.values),
            di1_.name: kwat.array.log_with_negative(di1_.values),
        },
        tp_ge_sa.index,
    )

    an_po_pa["Size"] = 8

    bo_ = kwat.iterable.check_is_in(
        an_po_pa.index.values,
        ch_.index[kwat.array.check_is_extreme(ch_.values, "<>", n_ex=5)].tolist() + pe_,
    )

    an_po_pa["Color"] = np.where(bo_, "#9017e6", "#20d0ba")

    an_po_pa["Opacity"] = np.where(bo_, 1, 0.48)

    an_po_pa["Annotate"] = bo_

    kwat.plot.plot_point(
        an_po_pa, title="Compare Differences", pa="{}{}.html".format(pa, na)
    )